In [1]:
!nvidia-smi

Sat Apr 27 22:58:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.76                 Driver Version: 550.76         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:0A:00.0  On |                  N/A |
|  0%   46C    P5             24W /  215W |     502MiB /   8192MiB |     31%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import opendatasets as od
import string
import random
import unicodedata
import os
import re

from collections import Counter
import nltk
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Normalize, ToTensor

In [3]:
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0)
print(device)

cuda:0


In [4]:
dataset_path = "Datasets/final.csv"
dataset = pd.read_csv(dataset_path)
dataset.head()

,Unnamed: 0,id,og,t
0,0,42928-1500614319216-63344,You do not meet a man but frowns:,Every man you meet these days is frowning.
1,1,42928-1500614326583-89821,our bloods No more obey the heavens than our...,Our bodies are in agreement with the planetar...
2,2,A-63849,But what's the matter?,What's wrong?
3,3,42930-1500614347266-80123,"His daughter, and the heir of's kingdom, whom...","The king wanted his daughter, the only heir to..."
4,4,42930-1500614355280-38326,she's wedded; Her husband banish'd; she impr...,"She's married, her husband is banished, she's..."


In [20]:
pairs = list(zip(dataset['og'], dataset['t']))
len(pairs)

51787

In [21]:
def normalizeString(s):
    sres=''
    for c in unicodedata.normalize('NFD', s):
        if unicodedata.category(c) != 'Mn':
            sres+=c
    
    sres = re.sub(r"([.!?])", r" \1", sres) 
    sres = re.sub(r"[^a-zA-Z!?]+", r" ", sres) 
    return sres.strip()

def createNormalizedPairs(pairs):
    initpairs = []
    for pair in pairs:
        s1, s2 = pair
        s1=normalizeString(s1.lower().strip())
        s2=normalizeString(s2.lower().strip())
        # print(len(s1), " ",len(s2))
        initpairs.append([s1,s2])
    # print(len(initpairs))
    return initpairs

In [22]:
MAX_LENGTH = 15

def filterpairs(initpairs):
    eng_prefixes = (
        "i am ", "i m ",
        "he is", "he s ",
        "she is", "she s ",
        "you are", "you re ",
        "we are", "we re ",
        "they are", "they re ",
        "i have ", "i ve ",
        "you have ", "you ve ",
        "we have ", "we ve ",
        "they have ", "they ve ",
        "is not ", "isn t ",
        "are not ", "aren t ",
        "was not ", "wasn t ",
        "were not ", "weren t ",
        "have not ", "haven t ",
        "has not ", "hasn t ",
        "had not ", "hadn t ",
        "will not ", "won t ",
        "would not ", "wouldn t ",
        "do not ", "don t ",
        "does not ", "doesn t ",
        "did not ", "didn t ",
        "can not ", "can t ",
        "could not ", "couldn t ",
        "should not ", "shouldn t ",
        "might not ", "mightn t ",
        "must not ", "mustn t "
    )
    pairs = []
    for pair in initpairs:
        if len(pair[0].split(" ")) < MAX_LENGTH and len(pair[1].split(" ")) < MAX_LENGTH and pair[0].lower().startswith(eng_prefixes):
            pairs.append(pair)
        
    print("Filtered pairs:", len(pairs))
    return pairs

In [23]:
pairs = createNormalizedPairs(pairs)
pairs = filterpairs(pairs)

Filtered pairs: 739


In [24]:
class Vocab:
    def __init__(self, name):
        self.name = name
        self.word2index={'SOS':0, 'EOS':1}
        self.index2word={0:'SOS', 1:'EOS'}
        self.word2count={}
        self.nwords = 2
        
    def buildVocab(self, s):
        for word in s.split(" "):
            if word not in self.word2index:
                self.word2index[word] = self.nwords
                self.index2word[self.nwords] = word
                self.word2count[word]=1
                self.nwords+=1
            else:
                self.word2count[word]+=1

In [ ]:
def get_input_ids(sentence, langobj):
    input_ids = []
    for word in sentence.split(" "):
        input_ids.append(langobj.word3index[word])
        
    if langobj.name=='shake':
        input_ids.append(langobj.word2index['EOS'])
    else:
        input_ids.insert(0,langobj.word2index['SOS'])
        input_ids.append(langobj.word2index['EOS'])
    return torch.tensor(input_ids)

In [ ]:
class customDataset(Dataset):
    def __init__(self):
        super().__init__()
        
    def __len__(self):
        return length
    
    def __getitem__(self, idx):
        t=pairs[idx][0]
        s=pairs[idx][1]